In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
import US_main, US_c, US_policy
os.chdir(d['curr'])

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.504840069, 1.394563144, 1.178137696, 1.055068673, 1.018706685, 1.018706685])
T_log = len(ν_log)
T = T_log+3
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
t0date = 2010 # calibration date
t0 = dates.get_loc(t0date) # index for year of calibration 


# "Simple" calibration:
α = .281 # capital income share

# Household types:
γj  = np.array([.056, 0.449, 0.307, 0.188])
ni = len(γj)-1
hours = np.array([1415.38, 2114.79, 2315.83, 2458.91]) # hours
wagerates = np.array([6.914, 14.88, 27.32, 62.26]) # wages
income = hours*wagerates
zxj = hours/hours[1:].mean()
zηj = income/income[1:].mean()
pj = np.array([0.385, 0.55, 0.652, 0.74])
μj = np.array([0.362, 0.526, 0.684, 0.756]) # voter turnout 


# other targets:
τ0 = .158 # target labor tax rate
RR = 39.4/50.1 # replacement rate
universalShare = 3.4/15.8 # share of ss going to universal
R0 = 2.443 


# Initial/baseline values:
ξ0 = .35
ρ0 = 1.2
ω0 = 1.5
# βj = np.ones(ni+1)

In [3]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'γj': γj, 'zxj': zxj, 'zηj': zηj, 'pj': pj, 'μj': μj, 'τ0': τ0, 'RR0': RR, 'UShare0': universalShare, 'R0': R0, 'ξ': ξ0, 'ρ': ρ0, 'ω': ω0}

In [4]:
m = US_main.Model(ni = ni, T = T, gridkwargs = {'glob_ns': 50}, **kwargs)

## 1. PEE class

The ```PEE``` class is used to identify policy functions. 
* It includes finite horizon methods, steady state methods, and infinite horizon methods (that assumes steady state for the terminal period only). 
* When identifying the policy functions, we can always identify the policy function in one of the following ways: 
    1. ```style = 'Vector'```: Optimization (on grid of $s_{t-1}$).
    2. ```style = 'ScalarLoop'```: loops through optimization for each node on the grid.
    3. ```style = 'GridSearch'```: Creates grid of $\tau$ (for each node $s_{t-1}$) and chooses the $\tau$ that minimizes distance to solution.
    4. ```style = 'Grid'```: Creates grid of $\tau$ and interpolates the solution.
    5. ```stlye = 'GridSC```: Creates grid of $\tau$ and interpolates solution by idenfitying sign changes in the objective function for adjacent nodes.
    
    Finally, the method ```style = 'Robust'``` applies ```GridSC``` to obtain an intial guess for the policy function and then applies the ```Vector``` method.
* The method ```FH``` returns dict of policy functions (over time). The default intial values can be accessed and adjusted through ```self.x0``` (dict over time), and the default "solution style" and other arguments can be passed through a dict (over time) of dicts (kwargs). The default styles are specified in ```self.kwargs_T``` and ```self.kwargs_t```. 

In [5]:
pee = US_policy.PEE(m)

*Solve terminal period with various styles:*

In [6]:
t = m.db['t'][-1]
θT, epsT, νT = m.db['θ'].iloc[-1], m.db['eps'].iloc[-1], ν[-1]
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'Vector', x0 = pee.x0[t])
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'ScalarLoop', x0 = pee.x0[t])
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'GridSearch', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'GridSC', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'Grid', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_T = pee.solve(θT, epsT, νT, t = 'T', style = 'Robust', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values

*Solve T-1 with various styles:*

In [7]:
t = m.db['t'][-2]
θt, epst, νt = m.db['θ'].iloc[-2], m.db['eps'].iloc[-2], ν[-2]
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'Vector', x0 = pee.x0[t])
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'ScalarLoop', x0 = pee.x0[t])
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'GridSearch', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'GridSC', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'Grid', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values
sol_t = pee.solve(θt, epst, νt, θp = θT, epsp = epsT, νp = νT, solp = sol_T, style = 'Robust', n = 1000, l = 0, u = 1) # add grid specifications for the search instead of initial values

*Return dict of policy functions using default methods:*

In [8]:
sol = pee.FH()

*Specify that $T-1$ should be solved with 'Robust' and otherwise standard settings:*

In [9]:
stdkwargs = pee.FH_kwargs
stdkwargs[t] = {'style': 'Robust', 'n': 100, 'l': 0, 'u': 1}
sol = pee.FH(pars = stdkwargs)

## 2. ESC class

*The ESC class is set up in a very similar way. The gridstyle ```GridSC``` applies the search for sign changes sequentially and is thus pretty slow*

In [10]:
esc = US_policy.ESC(m)

*Solve terminal period with various styles:*

In [11]:
t = m.db['t'][-1]
νT = ν[-1]
sol_ESC_T = esc.solve(νT, t = 'T', style = 'Vector', x0 = esc.x0[t])
sol_ESC_T = esc.solve(νT, t = 'T', style = 'ScalarLoop', x0 = esc.x0[t])
sol_ESC_T = esc.solve(νT, t = 'T', style = 'GridSearch', **esc.grids) # add grid specifications for the search instead of initial values - use standard grids from self.FH_grids
sol_ESC_T = esc.solve(νT, t = 'T', style = 'GridSC',**esc.grids) # add grid specs.
sol_ESC_T = esc.solve(νT, t = 'T', style = 'Grid', **esc.grids) # add grid specifications for the search instead of initial values
sol_ESC_T = esc.solve(νT, t = 'T', style = 'Robust', **esc.grids) # add grid specifications for the search instead of initial values

*Solve T-1 with various styles:*

In [12]:
t = m.db['t'][-2]
νt = ν[-2]
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'Vector', x0 = esc.x0[t])
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'ScalarLoop', x0 = esc.x0[t])
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'GridSearch', **esc.grids) # add grid specifications for the search instead of initial values
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'GridSC', **esc.grids) # add grid specifications for the search instead of initial values
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'Grid', **esc.grids) # add grid specifications for the search instead of initial values
sol_ESC_t = esc.solve(νt, νp = νT, solp = sol_ESC_T, style = 'Robust', **esc.grids) # add grid specifications for the search instead of initial values

*Return dict of policy functions using default methods:*

In [13]:
sol = esc.FH()

*Specify that $T-1$ should be solved with 'Robust' and otherwise standard settings:*

In [14]:
stdkwargs = esc.FH_kwargs
stdkwargs[t] = {'style': 'Robust'} | esc.grids # robust requires grid search parameters
sol = esc.FH(pars = stdkwargs)